In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import numpy as np
from acquire import remove_stopwords, basic_clean, tokenize , prep_and_split_data
from prepare_jag import prep_train, basic_clean3
import re
import os
from re import search
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow import keras

from nltk.corpus import stopwords
import nltk

import random
random.seed(333)

In [2]:
train_df, val_df, test_df = prep_and_split_data()

Number of rows in training set: 37931
Number of rows in validation set: 2108
Number of rows in test set: 2107


/usr/local/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [3]:
train_df

,case,student_notes,clean,targets
28846,7,35yo female presents with complaint of heavier...,35yo female presents complaint heavier period ...,"[female, weight gain, heavy periods or irregul..."
25230,5,Patient is a 26 yo F with no PMH who presents ...,patient 26 yo f no pmh presents follow visitin...,"[onset 5 years ago, female, no caffeine use, a..."
567,0,This is a 17 yo M presenting with 3 mo of hear...,17 yo presenting 3 mo heart pounding episodes ...,[family history of mi or family history of myo...
7527,3,Mr. Hamilton is a 35 year old male here with s...,mr hamilton 35 year old male stomach issues be...,[fhx of pud or family history of peptic ulcer ...
22673,5,Pt is a 26 year old woman with a 5 year histor...,pt 26 year old woman 5 year history palpations...,"[onset 5 years ago, female, no caffeine use, a..."
...,...,...,...,...
14864,4,45 Y/O F C/O NERVOUSNESS X FEW WEEKS STARTED S...,45 f c nervousness x weeks started suddenly sw...,"[lack of other thyroid symptoms, anxious or ne..."
28479,6,Kane Smith is a 17 yo male with a 1 day histor...,kane smith 17 yo male 1 day history sharp ches...,"[subjective fevers, male, 17 year, recent uppe..."
3293,2,Pt is a 44 F p/w CC of irregular periods\r\nSt...,pt 44 f p w cc irregular periods states last 3...,"[prior normal periods, last pap smear i year a..."
13993,3,35 yo male comes to clinic with a cc of a 2 mo...,35 yo male comes clinic cc 2 month hisotry gna...,[fhx of pud or family history of peptic ulcer ...


## Multi-label binarization
Let's preprocess our labels using the [StringLookup](https://keras.io/api/layers/preprocessing_layers/categorical/string_lookup) layer.

In [4]:
features = tf.ragged.constant(train_df['targets'].values)
lookup = tf.keras.layers.StringLookup(output_mode='multi_hot')
lookup.adapt(features)
vocab = lookup.get_vocabulary()

2022-03-04 22:31:06.499899: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
def invert_multi_hot(encoded_labels):
    '''Reverse a single multi-hot encoded label to a tuple of vocab terms.'''
    hot_indices = np.argwhere(encoded_labels == 1.0)[...,0]
    return np.take(vocab, hot_indices)

print('Vocabulary:\n')
print(vocab)

Vocabulary:

['[UNK]', 'female', 'nausea', '35 year', 'male', 'post prandial bloating or fullness with meals', 'nsaid use or nonsteroidal anti inflammatory drug use', 'no blood in stool', 'minimal to no change with tums', 'intermittent', 'getting worse or progressive or symptoms now daily', 'fhx of pud or family history of peptic ulcer disease', 'epigastric discomfort', 'duration 2 months', 'darker bowel movements', 'burning or gnawing or burning and gnawing', 'awakens at night', '2 to 3 beers a week', 'recent visit to emergency department with negative workup', 'onset 5 years ago', 'no illicit drug use', 'no chest pain', 'no caffeine use', 'increased stress', 'increased frequency recently', 'feels hot or feels clammy', 'fatigue or difficulty concentrating', 'episodes of heart racing', 'episodes last 15 to 30 minutes', 'episode of hand numbness or episode of finger numbness', 'associated throat tightness', 'associated sob or associated shortness of breath', 'associated nausea', 'associ

### Separate the individual targets from the label pool and then use it to represent a given label set with 0's and 1's

In [6]:
sample_label = train_df['targets'].iloc[0]
print(f'Original label: {sample_label}')

label_binarized = lookup([sample_label])
print(f'Label-binarized representation: {label_binarized}')

Original label: ['female', 'weight gain', 'heavy periods or irregular periods', 'last menstrual period 2 months ago', 'unprotected sex', 'fatigue', 'infertility hx or infertility history', '35 year', 'symptoms for 6 months']
Label-binarized representation: [[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


## Data preprocessing and [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) objects

In [7]:
train_df['clean'].apply(lambda x: len(x.split(" "))).describe()

count    37931.000000
mean        97.749941
std         16.350154
min          7.000000
25%         88.000000
50%        100.000000
75%        109.000000
max        150.000000
Name: clean, dtype: float64

In [8]:
max_seqlen = 98
batch_size = 128
padding_token = '<pad>'
auto = tf.data.AUTOTUNE

def make_dataset(dataframe, is_train=True):
    labels = tf.ragged.constant(dataframe['targets'].values)
    label_binarized = lookup(labels).numpy()
    dataset = tf.data.Dataset.from_tensor_slices(
        (dataframe['clean'].values, label_binarized)
    )
    dataset = dataset.shuffle(batch_size * 10) if is_train else dataset
    return dataset.batch(batch_size)

## Prepare the [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) objects.

In [9]:
train_dataset = make_dataset(train_df, is_train=True)
validation_dataset = make_dataset(val_df, is_train=False)
test_dataset = make_dataset(test_df, is_train=False)

## Preview the dataset

In [10]:
text_batch, label_batch = next(iter(train_dataset))

for i, text in enumerate(text_batch[:5]):
    label = label_batch[i].numpy()[None, ...]
    print(f'Student note: {text}')
    print(f'Targets: {invert_multi_hot(label[0])}')
    print(' ')

Student note: b'45 female presents episodes nervousness feels like much plate taking care work children mother started weeks ago getting worse no major life events professor english literatture affected sleep appetite decreased sexual desire also less usual feel sad kept concentraction interest no effect psychomotor skills no suicide ideations no known history psychiatric disorder no chest pain no palpitations no domestic abuse no allergies no medication fh father died 65'
Targets: ['female' 'weight stable' 'stress due to caring for elderly parents'
 'no depressed mood' 'lack of other thyroid symptoms' 'insomnia'
 'heavy caffeine use' 'decreased appetite' 'anxious or nervous' '45 year']
 
Student note: b'35 year old male presneted clinic epigastric pain past 2 months burning nature no radiating nothing makes worse intermitant occurs lasts 1 2 hours took tams relief pain pain progressed thru past 2 weeks occured 2 3 times nauea no vimiting color stool became dark also period no change b

## Vectorization
Vectorize the text to represent it as a quantitative value. We will use [TextVectorization layer](https://keras.io/api/layers/preprocessing_layers/text/text_vectorization)

In [11]:
# Get unique words in student notes.
vocabulary = set()
train_df['clean'].str.split().apply(vocabulary.update)
vocabulary_size = len(vocabulary)
print(vocabulary_size)

42065


## Now we create our vectorization layer and map() to the [tf.data.Datasets](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) created earlier.

In [27]:
text_vectorizer = layers.TextVectorization(
    max_tokens=vocabulary_size, ngrams=2, output_mode='tf_idf'
)

with tf.device('/CPU:0'):
    text_vectorizer.adapt(train_dataset.map(lambda text, label: text))

train_dataset = train_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto).prefetch(auto)
validation_dataset = validation_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto).prefetch(auto)
test_dataset = test_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto).prefetch(auto)

ValueError: in user code:

    File "/usr/local/anaconda3/lib/python3.8/site-packages/keras/engine/base_preprocessing_layer.py", line 118, in adapt_step  *
        self.update_state(data)
    File "/usr/local/anaconda3/lib/python3.8/site-packages/keras/layers/preprocessing/text_vectorization.py", line 431, in update_state  **
        self._lookup_layer.update_state(self._preprocess(data))
    File "/usr/local/anaconda3/lib/python3.8/site-packages/keras/layers/preprocessing/text_vectorization.py", line 520, in _preprocess
        raise ValueError(

    ValueError: When using `TextVectorization` to tokenize strings, the input rank must be 1 or the last shape dimension must be 1. Received: inputs.shape=(None, 42065) with rank=2


In [28]:
text_batch, label_batch = next(iter(train_dataset))

In [30]:
len(text_batch[0]), len(label_batch[0])

(42065, 132)

## Create a text classification model

In [31]:
def make_model():
    shallow_mlp_model = keras.Sequential(
        [
            layers.Dense(512, activation="relu"),
            layers.Dense(256, activation="relu"),
            layers.Dense(lookup.vocabulary_size(), activation="softmax"),
        ]
    )
    return shallow_mlp_model

## Train our model

In [32]:
epochs = 2

shallow_mlp_model = make_model()
shallow_mlp_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["categorical_accuracy"]
)

history = shallow_mlp_model.fit(
    train_dataset, validation_data=validation_dataset, epochs=epochs
)


def plot_result(item):
    plt.plot(history.history[item], label=item)
    plt.plot(history.history["val_" + item], label="val_" + item)
    plt.xlabel("Epochs")
    plt.ylabel(item)
    plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)
    plt.legend()
    plt.grid()
    plt.show()


plot_result("loss")
plot_result("categorical_accuracy")

Epoch 1/2
 87/297 [=======>......................] - ETA: 19s - loss: 0.0603 - categorical_accuracy: 0.3050

KeyboardInterrupt: 

## Evaluate the model

In [22]:
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 42065), dtype=tf.float32, name=None), TensorSpec(shape=(None, 132), dtype=tf.float32, name=None))>

In [23]:
validation_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 42065), dtype=tf.float32, name=None), TensorSpec(shape=(None, 132), dtype=tf.float32, name=None))>

In [24]:
test_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 42065), dtype=tf.float32, name=None), TensorSpec(shape=(None, 132), dtype=tf.float32, name=None))>

In [25]:
_, categorical_acc = shallow_mlp_model.evaluate(test_dataset)
print(f"Categorical accuracy on the test set: {round(categorical_acc * 100, 2)}%.")

17/17 [==============================] - 1s 28ms/step - loss: 8.9138e-05 - categorical_accuracy: 0.1761
Categorical accuracy on the test set: 17.61%.


In [26]:
# Create a model for inference.
model_for_inference = keras.Sequential([text_vectorizer, shallow_mlp_model])

# Create a small dataset just for demoing inference.
inference_dataset = make_dataset(test_df.sample(100), is_train=False)
text_batch, label_batch = next(iter(inference_dataset))
predicted_probabilities = model_for_inference.predict(text_batch)

# Perform inference.
for i, text in enumerate(text_batch[10:50]):
    label = label_batch[i].numpy()[None, ...]
    print(f"Student notes: {text}")
    print(f"Targets(s): {invert_multi_hot(label[0])}")
    predicted_proba = [proba for proba in predicted_probabilities[i]]
    all_labels = [
        x
        for _, x in sorted(
            zip(predicted_probabilities[i], lookup.get_vocabulary()),
            key=lambda pair: pair[0],
            reverse=True,
            
        )
    ][:]
    print(f"Predicted Targets(s): ({', '.join([label for label in all_labels])})")
    print(" ")

Student notes: b'mr chad hamilton 35 yo pmh chronic lower back pain treated motrin 10 yrs presents upper abd pain last 2 months increased frequency pain comes goes occurs twice daily used occur 1 2 times per week describes pain knawing nature denies radiation reports mild alleviation tums rates pain 5 10 pt denies correlation pain food intake ros significant nausea dark stools negative vomiting weight loss dyruria hematuria myalgias sob sensorineural changes pmh negative aside back pain fh significant uncle reportedly pain secondary bleeding ulcer'
Targets(s): ['male' '17 year' 'shortness of breath'
 'no hair changes or no nail changes or no temperature intolerance'
 'lightheaded' 'intermittent symptoms' 'heart pounding or heart racing'
 'few months duration' 'family history of thyroid disorder'
 'family history of mi or family history of myocardial infarction'
 'chest pressure' 'caffeine use' 'adderall use']
Predicted Targets(s): (intermittent symptoms, 17 year, few months duration, f